In [1]:
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
import base64
import hashlib
import hmac
import urllib.parse as urlparse
import json
import os
import dotenv

dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [2]:
def fetch_openstreetmap_image(lat, lon, zoom=15, size=(512, 512)):
    """
    Fetches a map image from OpenStreetMap centered at the given latitude and longitude.

    Parameters:
    - lat (float): Latitude of the location.
    - lon (float): Longitude of the location.
    - zoom (int): Zoom level (default is 15).
    - size (tuple): Size of the image (width, height) in pixels (default is 512x512).

    Returns:
    - Image object: An image of the map.
    """
    base_url = "https://static-maps.openstreetmap.de/staticmap.php"
    params = {
        "center": f"{lat},{lon}",
        "zoom": zoom,
        "size": f"{size[0]}x{size[1]}",
        "maptype": "mapnik"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        return img
    else:
        raise Exception(f"Error fetching map image: {response.status_code}")


In [12]:
og = pd.read_csv('../Data_Raw/project_tables/og_geocoded.csv')
og.head()

,project_number,project_link,project_type,project_description,last_reviewed,last_reviewed_meeting,review_type,project_location_clean,project_location_preload_geolocator,Coordinates,Coordinates_lat_lon,Coordinates_full
0,OG 25-038,/records-research/project-search/og-25-038-0,Commercial,Sign - Canal House,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1023-1029 31st street nw,"1023 31st street nw, washington, district of c...","1023, 31st Street Northwest, Peter Square, Geo...","(38.9032644, -77.060821)","{'place_id': 321633665, 'licence': 'Data © Ope..."
1,OG 25-037,/records-research/project-search/og-25-037-0,Georgetown,Shaw Field lights,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3700 o street nw,"3700 o street nw, washington, district of colu...","Georgetown University, 3700, O Street Northwes...","(38.90893925, -77.07457962060826)","{'place_id': 320719677, 'licence': 'Data © Ope..."
2,OG 25-036,/records-research/project-search/og-25-036-0,Residence,Egress window well on west elevation,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3321 q street nw,"3321 q street nw, washington, district of colu...","3321, Q Street Northwest, Georgetown, Ward 2, ...","(38.9108942, -77.06688820187202)","{'place_id': 323953263, 'licence': 'Data © Ope..."
3,OG 25-034,/records-research/project-search/og-25-034-0,Residence,Shutter and door replacements,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3306 o street nw,"3306 o street nw, washington, district of colu...","3306, O Street Northwest, Peter Square, George...","(38.907624049999995, -77.06655487369912)","{'place_id': 324062783, 'licence': 'Data © Ope..."
4,OG 25-033,/records-research/project-search/og-25-033-0,Residential,Landscape alterations and stair revision,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3114 r street nw,"3114 r street nw, washington, district of colu...","3114, R Street Northwest, Georgetown, Ward 2, ...","(38.91330825, -77.0637584339222)","{'place_id': 320720649, 'licence': 'Data © Ope..."


In [7]:
def get_image_link(input_url,secret):

    url = urlparse.urlparse(input_url)

    # We only need to sign the path+query part of the string
    url_to_sign = url.path + "?" + url.query

    # Decode the private key into its binary format
    # We need to decode the URL-encoded private key
    decoded_key = base64.urlsafe_b64decode(secret)

    # Create a signature using the private key and the URL-encoded
    # string using HMAC SHA1. This signature will be binary.
    signature = hmac.new(decoded_key, str.encode(url_to_sign), hashlib.sha1)

    # Encode the binary signature into base64 for use within a URL
    encoded_signature = base64.urlsafe_b64encode(signature.digest())

    original_url = url.scheme + "://" + url.netloc + url.path + "?" + url.query

    get_url = original_url + "&signature=" + encoded_signature.decode()
    return get_url

In [8]:
lat,lon = 38.907624049999995, -77.06655487369912
key = os.getenv('GOOGLE_API_KEY_2')
input_url = f"https://maps.googleapis.com/maps/api/streetview?size=400x400&location={lat},{lon}&fov=80&heading=70&pitch=0&key={key}"
secret = GOOGLE_API_KEY

request_url = get_image_link(input_url,secret)

request = requests.get(request_url)

if request.status_code == 200:
    image = Image.open(BytesIO(request.content))
    image.show()

In [13]:
og[og['project_type'] == 'Residential']

,project_number,project_link,project_type,project_description,last_reviewed,last_reviewed_meeting,review_type,project_location_clean,project_location_preload_geolocator,Coordinates,Coordinates_lat_lon,Coordinates_full
4,OG 25-033,/records-research/project-search/og-25-033-0,Residential,Landscape alterations and stair revision,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3114 r street nw,"3114 r street nw, washington, district of colu...","3114, R Street Northwest, Georgetown, Ward 2, ...","(38.91330825, -77.0637584339222)","{'place_id': 320720649, 'licence': 'Data © Ope..."
6,OG 25-030,/records-research/project-search/og-25-030-0,Residential,Front stone stoop repair and replacement,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),2819 p street nw,"2819 p street nw, washington, district of colu...","2819, P Street Northwest, Peter Square, George...","(38.9095261, -77.05799422332733)","{'place_id': 321320846, 'licence': 'Data © Ope..."
8,OG 25-025,/records-research/project-search/og-25-025-0,Residential,"Demo and reconfigure entry stairs, walls, and ...",2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1312 30th street nw,"1312 30th street nw, washington, district of c...","1312, 30th Street Northwest, Peter Square, Geo...","(38.90729545, -77.05940031827429)","{'place_id': 321559440, 'licence': 'Data © Ope..."
9,OG 25-024,/records-research/project-search/og-25-024-0,Residential,Front stoop and areaway expansion and railing ...,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1217 29th street nw,"1217 29th street nw, washington, district of c...","1217, 29th Street Northwest, Peter Square, Geo...","(38.905746, -77.05791314966635)","{'place_id': 320617805, 'licence': 'Data © Ope..."
17,OG 25-013,/records-research/project-search/og-25-013-0,Residential,Replace two patio doors on rear elevation,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3048 n street nw,"3048 n street nw, washington, district of colu...","3048, N Street Northwest, Peter Square, George...","(38.906674800000005, -77.0610026203689)","{'place_id': 320920437, 'licence': 'Data © Ope..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9435,OG 07-127,/records-research/project-search/og-07-127-0,Residential,"New house (on Williams-Addison House lot), gar...",2009-06-18T12:00:00Z,/records-research/record-cfa-actions/2009/06/c...,(CFA Meeting - OG Appendix),1645 31st street nw,"1645 31st street nw, washington, district of c...","1645, 31st Street Northwest, Peter Square, Geo...","(38.9117468, -77.06216092951931)","{'place_id': 321889536, 'licence': 'Data © Ope..."
9831,OG 08-030,/records-research/project-search/og-08-030,Residential,"Alterations: Replace railings, stairs, and cat...",2008-02-21T12:00:00Z,/records-research/record-cfa-actions/2008/02/c...,(CFA Meeting - OG Appendix),3700 o street nw,"3700 o street nw, washington, district of colu...","Georgetown University, 3700, O Street Northwes...","(38.90893925, -77.07457962060826)","{'place_id': 320719677, 'licence': 'Data © Ope..."
9855,OG 07-252,/records-research/project-search/og-07-252,Residential,"Rear and side window/door alterations, balconi...",2008-01-17T12:00:00Z,/records-research/record-cfa-actions/2008/01/c...,(CFA Meeting - OG Appendix),3045 n street nw,"3045 n street nw, washington, district of colu...","3045, N Street Northwest, Peter Square, George...","(38.9069817, -77.0608712)","{'place_id': 320666541, 'licence': 'Data © Ope..."
11138,OG 03-266,/records-research/project-search/og-03-266,Residential,New rowhouse building - lot 19,2003-09-18T12:00:00Z,/records-research/record-cfa-actions/2003/09/c...,(CFA Meeting - OG Appendix),1519 32nd street nw,"1519 32nd street nw, washington, district of c...","1519, 32nd Street Northwest, Georgetown, Ward ..."

In [16]:
location = og.loc[6,'project_location_preload_geolocator'].replace(', ',',').replace(' ','+')
location

'2819+p+street+nw,washington,district+of+columbia'

In [17]:
input_url = f"https://maps.googleapis.com/maps/api/streetview?size=400x640&location={location}&fov=50&pitch=10&key={key}"

request_url = get_image_link(input_url,secret)
req = requests.get(request_url)

if req.status_code == 200:
    image = Image.open(BytesIO(req.content))
    image.show()

In [18]:
import cv2
import numpy as np

In [19]:
image = cv2.imread('Screenshot 2024-12-12 185220.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.equalizeHist(gray)
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY, 11, 2)
# Optional morphological smoothing
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

edges = cv2.Canny(gray, 50, 150, apertureSize=3)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

# Then filter lines by slope and group them into vertical/horizontal sets
vertical_lines = []
horizontal_lines = []
for line in lines:
    x1, y1, x2, y2 = line[0]
    slope = (y2 - y1)/(x2 - x1 + 1e-6)
    if abs(slope) < 0.1:  # near horizontal
        horizontal_lines.append(line[0])
    elif abs(slope) > 10: # near vertical
        vertical_lines.append(line[0])

# From the vertical_lines and horizontal_lines, pick the ones that form the rectangle boundary.
# This might require sorting by position and picking the outermost lines.

# After identifying the four main lines, find their intersection points:
def line_intersection(l1, l2):
    # Compute intersection of two lines (each line defined by two points)
    x1,y1,x2,y2 = l1
    x3,y3,x4,y4 = l2
    denom = ((x1 - x2)*(y3 - y4) - (y1 - y2)*(x3 - x4))
    if denom == 0:
        return None
    intersectX = ((x1*y2 - y1*x2)*(x3 - x4) - (x1 - x2)*(x3*y4 - y3*x4)) / denom
    intersectY = ((x1*y2 - y1*x2)*(y3 - y4) - (y1 - y2)*(x3*y4 - y3*x4)) / denom
    return (intersectX, intersectY)

# Once you have the four intersection points, order them and perform the perspective transform as before.


In [ ]:
# Manually draw the points around the image
import cv2
import numpy as np

# Path to your input image
IMAGE_PATH = 'Screenshot 2024-12-12 185220.png'

# Global list to store clicked points
clicked_points = []

def select_point(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked_points.append((x, y))
        # Draw a small circle where the user clicked
        cv2.circle(img_display, (x,y), 5, (0,255,0), -1)
        cv2.imshow('Select 4 Corners', img_display)

def order_points(pts):
    # pts is a NumPy array of shape (4, 2)
    rect = np.zeros((4, 2), dtype="float32")

    # sum of points (x+y)
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right

    # difference of points (x-y)
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left

    return rect

# Load the image
image = cv2.imread(IMAGE_PATH)
if image is None:
    raise IOError("Could not open the image. Check the path and filename.")

# Create a copy for display during point selection
img_display = image.copy()
cv2.imshow('Select 4 Corners', img_display)
cv2.setMouseCallback('Select 4 Corners', select_point)

print("Please click on the four corners of the object in clockwise or counter-clockwise order.")

# Wait until we have 4 points
while True:
    if len(clicked_points) == 4:
        break
    if cv2.waitKey(1) & 0xFF == 27:
        # Press ESC to exit if needed
        break

cv2.destroyAllWindows()

# Convert the clicked points into a NumPy array
pts = np.array(clicked_points, dtype="float32")

# Order the points: top-left, top-right, bottom-right, bottom-left
ordered_points = order_points(pts)
(tl, tr, br, bl) = ordered_points

# Compute the new image width and height
widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
maxWidth = int(max(widthA, widthB))

heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
maxHeight = int(max(heightA, heightB))

# Destination points for the perspective transform
dst = np.array([
    [0, 0],
    [maxWidth - 1, 0],
    [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype="float32")

# Compute the perspective transform matrix and warp the image
M = cv2.getPerspectiveTransform(ordered_points, dst)
warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

# Display and save the resulting warped image
cv2.imshow('Warped Image', warped)
cv2.imwrite('corrected_image.jpg', warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Rectified image saved as 'corrected_image.jpg'.")


Please click on the four corners of the object in clockwise or counter-clockwise order.
Rectified image saved as 'corrected_image.jpg'.
